# Experiment with the MNIST data set

MNIST is a data set composed of hand-written digits that have been pre-processed to make the problem easier.  It's considered a 'toy' dataset now, but it was the original dataset that a lot of the earlier CNN work was done using.  The images are originally from scans of zip-codes on letters, and were provided by the US Post Office (see https://en.wikipedia.org/wiki/MNIST_database for more details).

**CUDA NOTE:** For this lab, we'll be using the Keras/Tensorflow deep learning library, which by default will try to use a GPU to accelerate the processing of your ANN via the CUDA toolchain.  That means that what computer you're running Jupyter on matters, since not all of our Lab machines have the same GPU.  It also means that if too many things are trying to use the GPU on the same machine at once, you'll run into problems (like running out of GPU memory).  Trying to run this code on your personal computer may go drastically slower, since there's a good chance you don't have a high-powered GPU along with a correctly configured driver/library stack to take advantage of it.

As a result, it's good practice to only have one Python kernel active, and to shut down the current Python kernel (in the 'Kernel' menu) before trying to use Tensorflow commands in a different notebook.

In [3]:
from aitk.utils import gallery, array_to_image
from aitk.networks import Network

import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

2024-03-25 15:20:01.468025: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Get the data
* Download the data
* Explore what you have

In [4]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [5]:
type(train_x)

numpy.ndarray

In [6]:
train_x.shape

(60000, 28, 28)

In [7]:
test_x.shape

(10000, 28, 28)

### Examining images
Let's look at a 'raw' vector, and then interpret it as them as images:

In [8]:
test_x[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  

In [9]:
images = [array_to_image(train_x[i]) for i in range(20)]

In [10]:
gallery(images)

0,1,2,3,4
5,6,7,8,9
10,11,12,13,14
15,16,17,18,19


In [11]:
train_y[:20]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9],
      dtype=uint8)

# Prepare the data for the network
* You may need to normalize the inputs so that they are in the range [0,1].
* You may need to convert the targets so that they are represented as 'one-hot' vectors when you are doing categorization (this means a vector with the number of elements equal to the number of categories, where the $i$'th element being 1 means it belongs to the $i$'th class).

###  Normalizing the input data

Start by finding the minimum value and the maximum value within your data set.

In [ ]:
min_input = train_x.min()
max_input = train_x.max()
print("range of input values is:", min_input, max_input)

The formula for normalizing your data is:

`(data - min_input)/(max_input - min_input)`

For most image data the min_input is 0 and the max_input is 255, as is the case here.  This simplifies the equation to:

`data/255`

_NOTE:_ be cautious, since if you do this on a per-element basis, you can have problems with e.g. the corners of the images always having a value of 0; for images like this, we want to normalize using the same range for every element, but for other types of data other strategies may be more appropriate


In [ ]:
train_x_normalized = train_x/255

In [ ]:
test_x_normalized = test_x/255

### Data sent into a Conv2D layer must have a depth
* This may require you to do a reshape command.
* For these black and white images there is only one channel of information.
* For color images there are typically 3 channels (Red, Green, Blue)

In [ ]:
train_x_normalized = train_x_normalized.reshape(60000,28,28,1)
test_x_normalized = test_x_normalized.reshape(10000,28,28,1)

### Target data

When doing a classification task we would typically convert the output into one-hot vectors. For some data sets you may not know how many classes there are.  You can put the training data into a set to find out how many unique classifications you have. 

In [ ]:
num_categories = len(set(train_y))

Then you can use the number of categories to produce one-hot vectors using `to_categorical`.

In [ ]:
train_y_category = to_categorical(train_y, num_categories)

In [ ]:
test_y_category = to_categorical(test_y, num_categories)

In [ ]:
train_y_category[0]

# Construct the model

This is just one possible configuration of layers to learn the MNIST data set.  Feel free to experiment with the number of filters, the filter size, and the layers themselves.

NOTE: In the final layer we are using an activation function called **softmax**.  This will output a pseudo-probability and is typically used in classification problems like the one we are trying to solve.  It should be paired with the loss function called **categorical_crossentropy**.

Also, this will produce some output that may look like 'warnings', but it's actually just information being produced by tensorflow.  It's *possible* that there might be actual issues in there, but more often than not you can ignore those messages safely.

In [ ]:
# Create a feed-forward network
neural_net = Sequential()

# add a convolutional layer, with 8 filters, each being 5x5; 
# uses the rectified linear activation function
# the 'input_shape' should match the shape of one training vector
# the 'name' is just for the humans looking at this, it doesn't impact the training process
neural_net.add(Conv2D(8,(5,5),activation="relu",name="conv1",input_shape=(28,28,1)))

# add a max-pooling layer with 2x2 
neural_net.add(MaxPooling2D(pool_size=(2,2),name="pool1"))

# the "flatten" step just re-arranges things, so all the nodes are lined up like they would be in an MLP
neural_net.add(Flatten(name="flatten"))

# add a fully-connected layer with 100 nodes using ReLU
neural_net.add(Dense(100, activation='relu',name="hidden"))

# add a fully-connected output layer using the 'softmax' activation
# the number of nodes should match the number of class labels (in this case, 10)
neural_net.add(Dense(10, activation='softmax',name="output"))

# this just prints out some info about the network architecture we've specified
neural_net.summary()

# Compile the model

Keras uses the method 'compile' to indicate that the network architecture is fully specified, and we're ready to set things up to actually do processing.

The 'optimizer' is what variant on gradient descent we'll be using, the 'loss' function is what defines the gradient, and the 'metrics' are what sort of scores we want it to report during the training process.

For our XOR networks we defined loss as sum-squared error.  However, for categorical data like handwritten digits it is better to use a different loss function called *categorical_crossentropy* (again, this is the loss function that is designed to be paired with the *softmax* output-layer activation function).  This interprets the outputs as representing pseudo-probabilities and forces them to sum to 1.0.  Thus the output from the network will reflect how likely it considers a particular input to be a member of one of the output categories.

The 'adam' optimizer is a version of SGD that uses ADAptive Momentum (hence the name), which means it's a bit less vulnerable to badly chosen hyperparameters like learning rate (here we'll leave things at their defaults, but you can specify lots of hyperparameters if you want, see https://keras.io/api/optimizers/adam/).

In [ ]:
neural_net.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['accuracy'])

# Create an aitk Network
This allows us to do more visualization of what is happening inside the network.  This is basically a front-end that we're adding on top of the Keras/Tensorflow based back-end, so it will take the network we've specified as its input.

In [ ]:
net = Network(neural_net)

Each layer may create a different range of values.  Configure the layers to a typical range to ensure that the layers will be properly displayed.

In [ ]:
net.set_config_layer("conv1", colormap=("gray", 0, 3))
net.set_config_layer("pool1", colormap=("gray", 0, 3))
net.set_config_layer("flatten",colormap=("gray", 0, 3))
net.set_config_layer("hidden", colormap=("gray", 0, 12))

The loop below shows you how the network performs on 10 sample images.  When you run this cell **prior** to training the output layer (at the top) has no clear winning categories before learning has occurred. However, if you re-run this cell **after** training you'll see that the network has correctly learned the classification for almost all of the images.

In [ ]:
from time import sleep

In [ ]:
for i in range(10,20):
    net.propagate(test_x_normalized[i])
    net.display(test_x_normalized[i])
    sleep(1.0)

# Train the model

In [ ]:
history = net.fit(train_x_normalized, # training examples
                  train_y_category,   # training labels
                  verbose=1,          # verbose output
                  validation_data=(test_x_normalized,  # validation examples
                                   test_y_category),   # validation labels
                  epochs=5)  # number of times to loop through the training set

# Inspect the feature maps

We can ask the network to propagate to specific layers and inspect the representations created there to try to understand how it has solved the problem.

Using one test image, find out what the maximum value is so that you can set up the color map properly. If you see a large red box in one of the visualized channels below, then use the image number for that test image to correct the color map.

In [ ]:
from math import ceil
for layer in ["conv1", "pool1"]:
    data = [net.propagate_to(test_x_normalized[49], layer, channel=channel)
            for channel in range(8)]
    largest = max([sublist.max() for sublist in data])
    net.set_config_layer(layer, colormap=("gray", 0, ceil(largest)))

Check out 10 different test images to see how each channel is representing them. Press the **enter** key to toggle thru the images.

In [ ]:



for test_case in range(40,50):
    c_images = [net.propagate_to(test_x_normalized[test_case], "conv1", "image", channel=channel)
                for channel in range(8)]
    c_bigger = [image.resize((200,200),resample=0) for image in c_images]
    original = test_x_normalized[test_case]
    gallery([original] + c_bigger, labels="channel{index}", gallery_shape=(9,1))
    input("press enter")

In [ ]:
for test_case in range(40,50):
    p_images = [net.propagate_to(test_x_normalized[test_case], "pool1", "image", channel=channel)
                for channel in range(8)]
    p_bigger = [image.resize((200,200),resample=0) for image in p_images]
    original = test_x_normalized[test_case]
    gallery([original] + p_bigger, labels="channel{index}", gallery_shape=(9,1))
    input("press enter")

# Examine the results
Check which inputs the network is getting wrong. Recall that there are 10 thousand test images.

In [ ]:
from numpy import argmax
outputs = net.predict(test_x_normalized)
answers = [argmax(output) for output in outputs]
targets = [argmax(target) for target in test_y_category]

In [ ]:
incorrect = [i for i in range(len(answers)) if answers[i] != targets[i]]
missed_target = [targets[i] for i in incorrect]
wrong_answer = [answers[i] for i in incorrect]
print("Number of incorrectly categorized images", len(missed_target))

We can use the `Counter` class from the `collections` library to find out which target is most commonly missed and which wrong answer is most commonly given.

In [ ]:
from collections import Counter

In [ ]:
t_ctr = Counter(missed_target)
t_ctr.most_common()

In [ ]:
a_ctr = Counter(wrong_answer)
a_ctr.most_common()

### Confusion Matrix
We can also build a **Confusion Matrix** to see what mistakes are being made.

This is a matrix (or 2D array) where the 'true' labels are arranged on the rows, and the 'predictions' are arranged on the columns (or vice-versa if you give the argument in the other order).  The count in a given cell is how many times that prediction-target combination occurred (i.e. how many times that prediction was given when that target _should_ have been given).

For more details on the library functions being used here, see:

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(targets, answers)
cm

In [ ]:
# there's also options for pretter formatting; this one uses a color-based heat-map by default
# there's a bunch of options to control the display in more detail if you want them
cm_plt = ConfusionMatrixDisplay(cm)
cm_plt.plot()

In [ ]:
images = [array_to_image(test_x[index]) for index in incorrect]
gallery(images, labels=wrong_answer)

### Interpretation

Notice the patterns here, and think about whether they make sense.  

You can look at either the rows or the columns, and they tell you slightly different things; one way is "when the network predicts X, what is the most likely 'true' answer?"  The other way is, "if the true answer is Y, what predictions are the most likely?"

Sometimes things are fairly symmetric, but not always, so it's worth looking at both.

For example, when I ran it I found that the number 9 tended to be confused with 4, 7, and 8; the number 7 tends to be confused with 2 and 9.

Is this reasonable?  Is the pattern of mistakes the same if you re-train the network?

Also note that based on the image visualization, some of the 'mistake' examples may look pretty silly to us, but others might be ambiguous even to a human (e.g. due to a bad scan).  How many of these errors are 'reasonable'?  Is it realistic to expect the network to do better than this?